In [11]:
import rasterio
import numpy as np
import random
import time
from numba import njit
import utils

### Inputs & Parameters

In [18]:
# Date
date_ini = '2017'
date_fin = '2017'

# Paths of Input files
urban_path_ini = 'Inputs/urban2017_roads.gif'
path_roads = 'Inputs/roads_17_mod.tif'
excluded_areas_path = 'Inputs/excluded.gif'
outside_boundaries_path = 'Inputs/outside_boundaries.tif'

# Objectifs Urbanization
objectif_urba =  24004    
edge_growth,spread_growth,road_growth,spont_growth = 93,4,1,2

### Reading inputs and estimating new urbanisation


In [19]:
# Urban Areas
urban_ini = rasterio.open(urban_path_ini).read(1)

# Indexes
x,y = urban_ini.shape
index_uni = np.array(list(range(x*y)))
index_double = [(a,b) for a in range(x) for  b in range(y)]
index_double_array = np.array(index_double)

# Roads
roads = rasterio.open(path_roads).read(1)

# Excluded Areas
excluded_areas = rasterio.open(excluded_areas_path).read(1)
excluded_areas_index = np.where(excluded_areas==255)
excluded_areas_index = [(a,b) for a,b in zip(excluded_areas_index[0],excluded_areas_index[1])]


dict_double_uni = dict(zip(index_double,index_uni))
dict_uni_double = dict(zip(index_uni,index_double))

index_exclu_1d = [dict_double_uni[x] for x in excluded_areas_index]
index_exclu_1d_array = np.array(index_exclu_1d)

# Outside Boundaries

raster = rasterio.open(outside_boundaries_path).read(1)
outside_boundaries = np.where(raster==0)
outside_boundaries_index = [(outside_boundaries[0][i],outside_boundaries[1][i]) for i in range(len(outside_boundaries[0]))]

# Raster Profile
raster_profile = rasterio.open(urban_path_ini).profile

# Ecluded + Outside Areas

exclu_outside_index = excluded_areas_index.copy()
exclu_outside_index.extend(outside_boundaries_index)



### Candidate Cells for Urbanization

In [20]:
candidate_cells = utils.create_growth_candidate_cells(urban_ini,roads,255,255,1,exclu_outside_index,0)

edge_growth_cells_da = utils.tuple_to_double_arr(candidate_cells[0])
spread_growth_cells_da = utils.tuple_to_double_arr(candidate_cells[1])
road_growth_cells_da = utils.tuple_to_double_arr(candidate_cells[2])
spont_growth_cells_da = utils.tuple_to_double_arr(candidate_cells[3])

### Create Order of Urbanization

In [21]:
nb_cells_edge = int(objectif_urba * edge_growth / 100)
nb_cells_road = int(objectif_urba * road_growth / 100)
nb_cells_spread = int(objectif_urba * spread_growth / 100)
nb_cells_spont = int(objectif_urba * spont_growth / 100)

order_urbanization = np.ones(nb_cells_edge)
order_urbanization = np.append(order_urbanization, 2 * np.ones(nb_cells_spread))
order_urbanization = np.append(order_urbanization, 3 * np.ones(nb_cells_road))
order_urbanization = np.append(order_urbanization, 4 * np.ones(nb_cells_spont))
order_urbanization = order_urbanization.astype('int8')
np.random.shuffle(order_urbanization)

In [22]:
evo_candidate_cells = urban_ini.copy()
evo_candidate_cells[edge_growth_cells_da] = 1
evo_candidate_cells[spread_growth_cells_da] = 2
evo_candidate_cells[road_growth_cells_da] = 3
evo_candidate_cells[spont_growth_cells_da] = 4

initial_urba = np.where(urban_ini.reshape(x*y)==255)[0]


## Simulation and Outputs

In [23]:
new_urban_cells = np.empty(len(order_urbanization))
new_urban_cells = utils.simulation_numba(evo_candidate_cells,initial_urba,order_urbanization,1,index_exclu_1d_array,index_double_array)
new_urb_tuple = index_double_array[new_urban_cells]

urb_fin = urban_ini.copy()
ea,eb = [a[0] for a in new_urb_tuple],[a[1] for a in new_urb_tuple]
urb_fin[ea,eb] = 255

out = 'Output/Simu_%s_%s.gif'%(date_ini,objectif_urba)
with rasterio.open(out,'w', **raster_profile) as dst:
    dst.write(urb_fin.reshape(1,1254,1548))